### Script purpose: Ming office title coding

1. General principles:
    - A comprehensive ontological structure of office title includes four parts: `Classification + Administrative Unit (optional) + Function (optional) + Title`
    - Each part corresponds to a table.
    - Separate `coding_value` and `raw_value`.
        - `raw_value`: the string appeared in original book text.
        - `coding_value`: the revised string that can be successfully coded.

2. Notes:
    - `Office title by LENGTH` table merges CBDB Ming office title with UCI table. Duplicates in CBDB table are removed in this table, i.e., this is the clean table we are going to use.

In [1]:
% matplotlib inline
import sqlite3
import pandas as pd
import networkx as nx
import xlrd
import matplotlib.pyplot as plt
import math
import warnings
from tqdm import tqdm
import re
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

### `c_office_chn` from UCI.

In [2]:
df_uci_office_ming=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSCmhbCk1B-9jjINMhy_VwikM6_Sn7bjdO7b_vaZJkVcYCCYlWVlhYVCFtAs0fPX-UEO62GWxaX1qAS/pub?gid=630627340&single=true&output=tsv',
                                    sep='\t')
df_uci_office_ming=df_uci_office_ming[['c_office_id（Dictionary Ser#)','Institution 1', 'Institution 2', 'Institution 3', 'c_office_chn']].rename(columns={'c_office_id（Dictionary Ser#)':'c_office_id'})
df_uci_office_ming['c_office_chn']=[s.replace('/', '') for s in df_uci_office_ming['c_office_chn']]
df_uci_office_ming.sample(3)

,c_office_id,Institution 1,Institution 2,Institution 3,c_office_chn
517,70640,中央中樞官署類 The Central Government,六部門 Six Ministries,禮部 The Ministry of Rites,左侍郎
499,1325,中央中樞官署類 The Central Government,六部門 Six Ministries,易州山廠 The Royal Charcoal Storehouse at Yizhou,易州山廠主事
2770,3047,地方軍事與治安機構類 Regional and Local Military Units,番夷都指揮使司門 Frontier Native Military Units,番夷衞指揮使司 Guard Command of Frontier Natives,指揮使


In [3]:
df_uci_office_ming['inst_1_chn']=[str(s).split()[0].replace('nan', '') for s in df_uci_office_ming['Institution 1']]
df_uci_office_ming['inst_2_chn']=[str(s).split()[0].replace('nan', '') for s in df_uci_office_ming['Institution 2']]
df_uci_office_ming['inst_3_chn']=[str(s).split()[0].replace('nan', '') for s in df_uci_office_ming['Institution 3']]
df_uci_office_ming['uci_value']=df_uci_office_ming['inst_1_chn']+df_uci_office_ming['inst_2_chn']+df_uci_office_ming['inst_3_chn']+df_uci_office_ming['c_office_chn']
df_uci_office_ming['c_office_id']=pd.to_numeric(df_uci_office_ming['c_office_id'], errors='coerce')
df_uci_office_ming.drop(['inst_1_chn', 'inst_2_chn', 'inst_3_chn', 'Institution 1', 'Institution 2', 'Institution 3', 'c_office_chn'], axis=1, inplace=True)

In [4]:
df_uci_office_ming[df_uci_office_ming['c_office_id'].duplicated()]

,c_office_id,uci_value
1130,71508.0,中央輔佐官署類秘書門翰林院直文淵閣侍講學士
1195,71503.0,中央輔佐官署類考官門會試官知貢舉官
1219,72165.0,中央輔佐官署類考官門鄉試官順天同考官
1282,NaN,京衛京營與中央軍事官署類京營門京營京營總兵官
2314,71504.0,地方官署類省官門行中書省理問所知事
2718,71274.0,地方軍事與治安機構類招討經略安撫使門宣撫司宣撫司經歷
2821,NaN,文武散階勛爵類勛爵門伯平涼伯
2842,NaN,文武散階勛爵類勛爵門伯新城伯
2862,NaN,文武散階勛爵類勛爵門伯永定伯
2882,NaN,文武散階勛爵類勛爵門伯鎮遠伯


In [5]:
df_uci_office_ming.drop(df_uci_office_ming[df_uci_office_ming['c_office_id'].duplicated()].index, inplace=True)
df_uci_office_ming.set_index('c_office_id', inplace=True)
df_uci_office_ming.sample(3)

,uci_value
c_office_id,
71900.0,皇族宮廷類宗室門藩王魯王
2422.0,南京官署類南京司法監察門南京大理寺/左寺正
1767.0,中央輔佐官署類僧道官門道錄司右玄義


### `c_office_chn` from CBDB uncleaned, and merge with UCI.

In [6]:
conn = sqlite3.connect('../../SQL/20170424CBDBauUserSqlite.db')
df_cbdb_office_ming=pd.read_sql_query("SELECT * FROM OFFICE_CODES", conn)[pd.read_sql_query("SELECT * FROM OFFICE_CODES", conn).c_dy==19].set_index('c_office_id')
df_cbdb_office_ming.sample(3)

,tts_sysno,c_dy,c_office_pinyin,c_office_chn,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,c_notes,c_category_1,c_category_2,c_category_3,c_category_4,c_office_id_old
c_office_id,,,,,,,,,,,,,,,,
70717,17262,19.0,nan li ji xun si lang zhong,南吏稽勲司郎中,None,None,Director of the Bureau of Records of the Southern,None,NaN,None,None,Director of the Bureau of Records of the Southern,None,None,None,0
70097,16610,19.0,ce feng gui fei zheng shi,冊封貴妃正使,None,None,Principal Commissioner to Confer the Title of ...,None,NaN,None,None,Principal Commissioner to Confer the Title of ...,None,None,None,0
70311,16824,19.0,fu qian hu,副千戶,None,None,Battalion Vice Commander,None,NaN,None,None,Battalion Vice Commander,None,None,None,0


In [7]:
for index in df_uci_office_ming.index:
    if index in df_cbdb_office_ming.index:
        df_uci_office_ming.loc[index, 'cbdb_value']=df_cbdb_office_ming.loc[index, 'c_office_chn']
        df_uci_office_ming.loc[index, 'tts_sysno']=df_cbdb_office_ming.loc[index, 'tts_sysno']
        df_uci_office_ming.loc[index, 'c_office_pinyin']=df_cbdb_office_ming.loc[index, 'c_office_pinyin']
        df_uci_office_ming.loc[index, 'c_office_pinyin_alt']=df_cbdb_office_ming.loc[index, 'c_office_pinyin_alt']
        df_uci_office_ming.loc[index, 'c_office_chn_alt']=df_cbdb_office_ming.loc[index, 'c_office_chn_alt']
        df_uci_office_ming.loc[index, 'c_office_trans']=df_cbdb_office_ming.loc[index, 'c_office_trans']
        df_uci_office_ming.loc[index, 'c_office_trans_alt']=df_cbdb_office_ming.loc[index, 'c_office_trans_alt']
        df_uci_office_ming.loc[index, 'c_source']=df_cbdb_office_ming.loc[index, 'c_source']
        df_uci_office_ming.loc[index, 'c_pages']=df_cbdb_office_ming.loc[index, 'c_pages']
        df_uci_office_ming.loc[index, 'c_notes']=df_cbdb_office_ming.loc[index, 'c_notes']
        df_uci_office_ming.loc[index, 'c_category_1']=df_cbdb_office_ming.loc[index, 'c_category_1']
        df_uci_office_ming.loc[index, 'c_category_2']=df_cbdb_office_ming.loc[index, 'c_category_2']
        df_uci_office_ming.loc[index, 'c_category_3']=df_cbdb_office_ming.loc[index, 'c_category_3']
        df_uci_office_ming.loc[index, 'c_category_4']=df_cbdb_office_ming.loc[index, 'c_category_4']
        df_uci_office_ming.loc[index, 'c_office_id_old']=df_cbdb_office_ming.loc[index, 'c_office_id_old']
df_uci_office_ming.loc[index, 'c_dy']=19

In [8]:
df_office_ming_merged=df_uci_office_ming
df_office_ming_merged.sample(3)

,uci_value,cbdb_value,tts_sysno,c_office_pinyin,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,c_notes,c_category_1,c_category_2,c_category_3,c_category_4,c_office_id_old,c_dy
c_office_id,,,,,,,,,,,,,,,,,
1567.0,中央輔佐官署類寺監門光祿司卿,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
831.0,皇族宮廷類女官門尚功局尚功,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70960.0,中央輔佐官署類秘書門通政使司通政使司參議,通政使司參議,17473.0,tong zheng shi si can yi,tong zheng si shi can yi;tong zheng si can yi,通政司試參議;通政司參議,Assistant Administration Commissioner in the O...,NaN,NaN,NaN,NaN,Assistant Administration Commissioner in the O...,NaN,NaN,NaN,0.0,NaN


### Coding `c_office_chn`.

In [9]:
df_adm=pd.read_csv('../data_output/C_OT_ADM.tsv', sep='\t').set_index('c_ot_adm_id')
df_cls=pd.read_csv('../data_output/C_OT_CLS.tsv', sep='\t').set_index('c_ot_cls_id')
df_tit=pd.read_csv('../data_output/C_OT_TIT.tsv', sep='\t').set_index('c_ot_tit_id')
df_func=pd.read_csv('../data_output/C_OT_FUNC.tsv', sep='\t').set_index('c_ot_func_id')

In [10]:
df_tit.sample(3)

,c_ot_tit_chinm,value_to_run,c_ot_tit_desc,c_ot_tit_start,c_ot_tit_end
c_ot_tit_id,,,,,
1021,薊王,2.0,NaN,NaN,NaN
805,主簿,1.0,NaN,NaN,NaN
412,恭順王,2.0,NaN,NaN,NaN


In [11]:
df_office_ming_merged['c_ot_coding']=df_office_ming_merged['uci_value']

In [ ]:
# Replace titles (only one title in an office title string).
for ming_ot_index in tqdm(df_office_ming_merged.index):
    ming_ot = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']
    ming_ot_done=[]
    for tit_index in df_tit.index:
        tit=df_tit.loc[tit_index, 'c_ot_tit_chinm']
        if ming_ot.endswith(tit) and ming_ot not in ming_ot_done:
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_tit_chinm']=tit
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=ming_ot.replace(tit, 'T'+str(tit_index))
            ming_ot_done.append(ming_ot)
df_office_ming_merged.sample(3)

100%|██████████| 4304/4304 [02:26<00:00, 29.31it/s]


,uci_value,cbdb_value,tts_sysno,c_office_pinyin,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,c_notes,c_category_1,c_category_2,c_category_3,c_category_4,c_office_id_old,c_dy,c_ot_coding,c_ot_tit_chinm
c_office_id,,,,,,,,,,,,,,,,,,,
70931.0,司法監察機構類監察門總督巡撫官兩廣總督,提督兩廣軍務兼巡撫,17444.0,ti du liang guang jun wu jian xun fu,None,None,Superintendent of Military Affairs and Grand C...,None,NaN,NaN,NaN,Superintendent of Military Affairs and Grand C...,NaN,NaN,NaN,0.0,NaN,司法監察機構類監察門T1296巡撫官兩廣T1296,總督
555.0,皇族宮廷類宦官門外差宦官靈臺看時近侍,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,皇族宮廷類宦官門外差宦官靈臺看時T1321,近侍
113.0,皇族宮廷類宗室門王府長史司典簿,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,皇族宮廷類宗室門王府長史司T1319,典簿


In [ ]:
# Replace Classifications (can be multiple units in an office title string).
for ming_ot_index in tqdm(df_office_ming_merged.index):
    cls_list=[]
    for cls_index in df_cls.index:
        cls=df_cls.loc[cls_index, 'c_ot_cls_chinm']
        c_ot_coding = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']
        if cls in c_ot_coding:
            cls_list.append(cls)
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding.replace(cls, 'C'+str(cls_index))
    if cls_list!=[]:
        df_office_ming_merged.loc[ming_ot_index, 'c_ot_cls_chinm']='#'.join(cls_list)
df_office_ming_merged.sample(3)

  4%|▍         | 189/4304 [00:01<00:22, 185.69it/s]

In [14]:
# Replace admin units (can be multiple units in an office title string).
for ming_ot_index in tqdm(df_office_ming_merged.index):
    adm_list=[]
    for adm_index in df_adm.index:
        adm=df_adm.loc[adm_index, 'c_ot_adm_chinm']
        c_ot_coding = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']
        if adm in c_ot_coding:
            adm_list.append(adm)
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding.replace(adm, 'A'+str(adm_index))
    if adm_list!=[]:
        df_office_ming_merged.loc[ming_ot_index, 'c_ot_adm_chinm']='#'.join(adm_list)
df_office_ming_merged.sample(3)

100%|██████████| 4304/4304 [02:05<00:00, 34.18it/s]


,uci_value,cbdb_value,tts_sysno,c_office_pinyin,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,...,c_category_1,c_category_2,c_category_3,c_category_4,c_office_id_old,c_dy,c_ot_coding,c_ot_tit_chinm,c_ot_cls_chinm,c_ot_adm_chinm
c_office_id,,,,,,,,,,,,,,,,,,,,,
71973.0,皇族宮廷類宗室門藩王壽王,壽王,18486.0,shou wang,None,None,[Not Yet Translated],爵,NaN,NaN,...,[Not Yet Translated],NaN,NaN,NaN,0.0,NaN,C23C55藩王T975,壽王,皇族宮廷類#宗室門,NaN
71109.0,地方軍事與治安機構類地區軍官門甘肅軍官協守副總兵,協總兵,17622.0,xie zong bing,None,None,Regional Associate Commander,None,NaN,NaN,...,Regional Associate Commander,NaN,NaN,NaN,0.0,NaN,C1C14甘肅軍官協守T125,副總兵,地方軍事與治安機構類#地區軍官門,NaN
173.0,皇族宮廷類宗室門王相府典膳正,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,C23C55王相府T160,典膳正,皇族宮廷類#宗室門,NaN


In [15]:
# Replace functional units (can be multiple units in an office title string).
for ming_ot_index in tqdm(df_office_ming_merged.index):
    func_list=[]
    for func_index in df_func.index:
        func=df_func.loc[func_index, 'c_ot_func_chinm']
        c_ot_coding = df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']
        if func in c_ot_coding:
            func_list.append(func)
            df_office_ming_merged.loc[ming_ot_index, 'c_ot_coding']=c_ot_coding.replace(func, 'F'+str(func_index))
    if func_list!=[]:
        df_office_ming_merged.loc[ming_ot_index, 'c_ot_func_chinm']='#'.join(func_list)
df_office_ming_merged.sample(3)

100%|██████████| 4304/4304 [00:35<00:00, 121.69it/s]


,uci_value,cbdb_value,tts_sysno,c_office_pinyin,c_office_pinyin_alt,c_office_chn_alt,c_office_trans,c_office_trans_alt,c_source,c_pages,...,c_category_2,c_category_3,c_category_4,c_office_id_old,c_dy,c_ot_coding,c_ot_tit_chinm,c_ot_cls_chinm,c_ot_adm_chinm,c_ot_func_chinm
c_office_id,,,,,,,,,,,,,,,,,,,,,
70106.0,中央輔佐官署類秘書門外交使節冊封親王副使,冊封親王副使,16619.0,ce feng qin wang fu shi,None,None,Assistant Commissioner to Confer the Title of ...,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,C8C48外交使節冊封親王T1018,副使,中央輔佐官署類#秘書門,NaN,NaN
70547.0,地方官署類省官門各道監軍參議,監軍參議,17092.0,jian jun can yi,None,None,Army-inspecting Assistant Administration Commi...,None,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,C17C54各道監軍T871,參議,地方官署類#省官門,NaN,NaN
71471.0,地方軍事與治安機構類地區軍官門掛印將軍征虜左副副將軍,征虜左副副將軍,17984.0,zheng lu zuo fu fu jiang jun,None,None,Left Vice General of the Expeditions against N...,None,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,C1C14掛印T800征虜左副副T800,將軍,地方軍事與治安機構類#地區軍官門,NaN,NaN


In [40]:
for index in tqdm(df_office_ming_merged.index):
    c_ot_coding=df_office_ming_merged.loc[index, 'c_ot_coding']
    if re.sub(r'A|C|T|F|\d', '', string=c_ot_coding)!='':
        df_office_ming_merged.loc[index, 'pass']='F'
    else:
        df_office_ming_merged.loc[index, 'pass']='T'

100%|██████████| 4304/4304 [00:05<00:00, 795.86it/s]


In [41]:
df_office_ming_merged.to_excel('../data_output/ming_office_title_merged_coding.xlsx', encoding='utf8')